# LangChain Demo

In [14]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

load_dotenv(".env_demo")
chat_model = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
    openai_api_base="https://api.siliconflow.cn/",
    openai_api_key=os.getenv("SILICONFLOW_API_KEY")
)

### Model

In [11]:
output = chat_model.invoke("介绍 k8s。")
print(output.content)


好的，没问题。我们来详细介绍一下 Kubernetes，也就是通常所说的 K8s。

想象一下，你正在开发一个应用程序，并且你想把它部署到生产环境。

*   **传统方式**：你可能需要配置物理服务器或虚拟机，安装操作系统、数据库、中间件等，然后打包你的应用，小心翼翼地部署过去，检查环境，测试连接。每次更新版本，可能都需要重复这些步骤，还担心会影响到线上服务。
*   **使用Docker等容器技术后**：你将应用打包进一个轻量级的、可移植的“集装箱”（Container）里，称为 Docker镜像。部署时，你只需要一个相同的容器镜像，让运行环境拉取即可。但很快，你就会遇到问题：如何统一管理大量这样的容器？它们如何启动？如何健康检查？一个容器挂了，怎么自动重启？灰度发布怎么弄？如果希望多个容器协同工作处理并行请求呢？
*   **这时 K8s 登场了！** Kubernetes 就像是一个专门管理 Docker （或其他符合标准的容器）的“**指挥官**”、“**园丁**”或者更准确地说是一个 “Platform as a Service”（PaaS）。

**核心理念 - 自动化、编排和管理**：K8s 的主要目标是自动化应用程序的部署、扩展和管理过程，特别是在由大量网络节点组成的集群上运行多个容器化应用时。

**以下是 K8s 的一些核心特性和发挥作用的方式：**

1.  **声明式配置**：你告诉 K8s “**我想要什么**”，而不是“**我该怎么做**”。比如，你指定需要 3 个副本的 Web 应用实例，以及存储 100GB 的持久化存储。K8s 会自动帮你检查现状和目标状态的差异，然后自动采取措施来缩小或扩大差距（比如启动、杀死容器，申请或释放存储）。它保持这个“想要”的状态。
2.  **容器部署与编排**：K8s 能够理解你的容器依赖关系。它可以将军你打包的容器（称为 Pod）安排到合适的节点（机器）上运行，并且确保它们能够有效地通信。
3.  **弹性伸缩**：
    *   **手动伸缩**：管理员可以手动增加或减少 Pod 的数量。
    *   **自动伸缩**：K8s 可以根据 CPU 使用率、内存使用率或其他自定义指标，自动增加或减少副本数量，以应对流量高峰或节省资源。这被称为 HPA。
4.  **服务发现和负载均衡**：在

### Prompt Template

In [15]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的AI助手，用中文简洁回答技术问题"),
    ("human", "列出3个使用{technology}的好处：")
])

chain = prompt | chat_model
response = chain.invoke({"technology": "Kubernetes(k8s)"})
print(response.content)


好的，使用Kubernetes (k8s) 的三个主要好处是：

1. **自动化部署和管理：** 通过声明式配置和编排，Kubernetes能自动部署、更新、回滚应用，并管理容器的生命周期、网络和存储，极大地简化了运维工作。
2. **弹性伸缩：** 能够根据负载自动调整应用的副本数量，实现水平扩展，优化资源利用率，并保证应用的服务质量。
3. **高可用性：** Kubernetes能够在节点故障或容器崩溃时自动替换故障实例，并将流量重新路由到健康的实例，确保应用整体保持可用状态。


### Output Parser

In [15]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import json

class ProductReview(BaseModel):
    product_name: str = Field(description="The name of the product being reviewed")
    rating: int = Field(description="The rating given to the product, out of 5")
    review_text: str = Field(description="The text of the review")
    pros: str = Field(description="The positive aspects mentioned in the review")
    cons: str = Field(description="The negative aspects mentioned in the review")

parser = JsonOutputParser(pydantic_object=ProductReview)

prompt = PromptTemplate(
    template="Provide a detailed product review based on the user's input.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

review_query = "请评论最新的智能手机型号 iPhone 14。"

chain = prompt | chat_model | parser

parsed_review = chain.invoke({"query": review_query})

print(json.dumps(parsed_review, indent=2, ensure_ascii=False)) 

{
  "product_name": "iPhone 14",
  "rating": 4,
  "review_text": "iPhone 14 是一款功能全面的智能手机，相比前代产品有所升级。设计上更轻薄，质感提升明显，A15 仿生芯片性能出色，无论是处理速度还是图形能力都处于领先水准。相机系统更加出色，尤其是低光环境中表现有很大提升，视频拍摄更加稳定。USB-C 接口的加入解决了前代需要额外适配器的问题，使用体验更加便捷。iOS 系统流畅性依旧优秀，但部分用户反映系统偶发 Bug，比如地图导航错误或文件下载失败等问题。",
  "pros": "1. A15 芯片性能强劲，用户体验流畅；2. 相机系统尤其夜拍和视频录制进步明显；3. USB-C 接口设计改善了前代的充电和数据传输体验；4. 设计更加精致，机身更轻便；5. iOS 系统生态稳定，与 App 兼容性良好。",
  "cons": "1. 缺少灵动岛功能增强体验，部分功能受限；2. 虽然 B 决版采用 USB-C，但充电速度仍较慢；3. 存储容量规划不合理，极大文件占用空间大；4. 系统偶发 Bug，部分问题反馈解决不及时；5. 屏幕刷新率仅 60Hz，不如 Pro 型号流畅。"
}


### Document Loader

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from tabulate import tabulate

loader = CSVLoader(file_path="books_output.csv", source_column="Book Title")
data = loader.load()
table_data = []
headers = ["Bool Title"]

for document in data:
    table_data.append([document.metadata['source']])

print(tabulate(table_data, headers=headers, tablefmt="grid"))

+-------------------------+
| Bool Title              |
+=========================+
| The Silent Patient      |
+-------------------------+
| Atomic Habits           |
+-------------------------+
| Educated                |
+-------------------------+
| Project Hail Mary       |
+-------------------------+
| Where the Crawdads Sing |
+-------------------------+


### Text Splitters

In [7]:
from langchain_text_splitters import HTMLHeaderTextSplitter

with open("sample_document.html", "r", encoding="utf-8") as file:
    html_string = file.read()

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
chunks = html_splitter.split_text(html_string)
print(f"Total chunks created: {len(chunks)}")
print(chunks)

Total chunks created: 4
[Document(metadata={'Header 1': '示例书目表（CSV）'}, page_content='示例书目表（CSV）'), Document(metadata={'Header 1': '示例书目表（CSV）'}, page_content='data URI 包含下面 CSV 内容，可用来下载 books_output.csv  \nDownload CSV (保存为 books_output.csv)  \n说明：代码示例中使用的 ，因此 CSV 必须包含名为 的列。下面表格就是 CSV 的可视化展示。  \nsource_column="Book Title"  \nBook Title  \nBook Title  \nAuthor  \nThe Hobbit  \nJ.R.R. Tolkien  \nDune  \nFrank Herbert  \n1984  \nGeorge Orwell  \nTo Kill a Mockingbird  \nHarper Lee  \nThe Lord of the Rings  \nJ.R.R. Tolkien'), Document(metadata={'Header 1': '示例书目表（CSV）', 'Header 2': 'CSV 原始内容（可复制）'}, page_content='CSV 原始内容（可复制）'), Document(metadata={'Header 1': '示例书目表（CSV）', 'Header 2': 'CSV 原始内容（可复制）'}, page_content="Book Title,Author\nThe Hobbit,J.R.R. Tolkien\nDune,Frank Herbert\n1984,George Orwell\nTo Kill a Mockingbird,Harper Lee\nThe Lord of the Rings,J.R.R. Tolkien  \n使用方法：点击上面的“Download CSV”保存为 ，然后运行你的 Python 脚本，代码会把 列作为每个 Document 的 。  \nbooks_output.csv  \nBook Title  \nmetadata[

### Memory

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content="You are a career coach chatbot, helping users with their career goals. Always respond in Chinese."
    ),  # The persistent system prompt
    MessagesPlaceholder(
        variable_name="chat_history"
    ),  # Where the memory will be stored
    HumanMessagePromptTemplate.from_template(
        "{human_input}"
    ),  # Where the human input will be injected
])

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_llm_chain = LLMChain(
    llm = chat_model,
    prompt = prompt,
    memory = memory,
    verbose = False
)


response = chat_llm_chain.predict(human_input="I'm thinking about switching careers to data science.")
print(response)

response = chat_llm_chain.predict(human_input="What skills do I need to develop for a data science role?")
print(response)

response = chat_llm_chain.predict(human_input="I have started learning Python. What should I focus on next?")
print(response)



很高兴听到你对转行到数据科学感兴趣！这个领域确实充满机遇，而且正在迅速发展。在你决定投入时间和精力去学习之前，我们可以一起评估一下几个重要的方面，帮助你更好地规划转型路径：

### 1. **自我评估**
   - **兴趣与热情**：你对手数据分析、编程、机器学习等方向是否有浓厚的兴趣？
   - **基础知识**：你是否具备一定的数学、统计学、计算机科学背景？这些是数据科学的核心基础。
   - **技能差距**：如果你现在的工作涉及相关技能，比如Python、SQL、数据可视化工具等，可以针对性地填补差距；如果是完全空白，可能需要系统性学习。

### 2. **行业现状**
   - **市场需求**：数据科学是当前热门领域，许多行业（如金融、电商、科技等）都有对数据科学家和分析师的需求。
   - **薪资水平**：数据科学相关职位通常有较高的薪资待遇，但这也取决于个人技能和经验。

### 3. **学习路径**
   - **在线课程**：平台如Coursera、edX、Udemy、DataCamp等提供丰富的数据科学课程。
   - **项目实践**：通过实际项目来应用所学知识，GitHub是一个很好的分享和展示自己项目的平台。
   - **社区参与**：加入数据科学社区（如Kaggle、Reddit的r/datascience等），参与讨论和项目合作。

### 4. **职业过渡**
   - **建立作品集**：通过项目经验、实习或志愿者活动来积累实践经验，作品集是求职时的重要工具。
   - **求职准备**：复习简历、准备面试，熟悉常见的面试问题和技术评估。

### 5. **心理准备**
   - **长期投入**：转行是一个过程，可能需要几个月甚至几年的时间来掌握所需技能。
   - **耐心与毅力**：过程中会有挫折和困难，保持积极性和坚持是关键。

如果你对这些方面有了更清晰的认识，并且愿意投入时间和努力去学习，那么转行到数据科学是一个值得考虑的选择。

如果你有更具体的问题或需要进一步的建议，我们可以继续深入探讨！😊

太好了，我很高兴你对数据科学角色的技能发展感兴趣！数据科学是一个多学科领域，涉及编程、统计学、机器学习和数据可视化等方面。掌握这些技能是转行成功的关键。下面，我来帮你梳理一下核心技能，并提供一些实用建议。我

### Agent Program

In [16]:
from langchain import PromptTemplate
from langchain.tools import StructuredTool, DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel

class TaskStatusRetriever:
    def __init__(self, task_database):
        self.task_database = task_database

    def get_status(self, task_id: str) -> str:
        return self.task_database.get(task_id, "Task not found")

task_database = {
    "task1": "In Progress",
    "task2": "Completed",
    "task3": "Not Started",
}

task_status_retriever = TaskStatusRetriever(task_database)

def task_status_lookup(task_id: str) -> str:
    return task_status_retriever.get_status(task_id)

class TaskStatusInput(BaseModel):
    task_id: str

task_status_tool = StructuredTool.from_function(
    name="TaskStatusLookup",
    description="Look up the status of a task by its ID.",
    func=task_status_lookup,
    args_schema=TaskStatusInput
)

duckduckgo_search = DuckDuckGoSearchResults()

def search_docs(query: str) -> str:
    return duckduckgo_search.run(query)

class DocsSearchInput(BaseModel):
    query: str

docs_search_tool = StructuredTool.from_function(
    name="DocsSearch",
    description="Search for relevant documentation online.",
    func=search_docs,
    args_schema=DocsSearchInput
)

tools = [
    task_status_tool,
    docs_search_tool
]

prompt_template = """
You are a project assistant. You can look up the status of
tasks in a project management system and search for relevant
documentation online.
Respond based on the user's input using the appropriate tools.

User's input: {input}

{agent_scratchpad}
"""
prompt = PromptTemplate.from_template(prompt_template)

chat_model = ChatOpenAI(
    model="deepseek-ai/DeepSeek-V3",
    openai_api_base="https://api.siliconflow.cn/",
    openai_api_key=os.getenv("SILICONFLOW_API_KEY")
)

agent = create_tool_calling_agent(chat_model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "What's the status of task1?"})
print(response['output'])

response = agent_executor.invoke({"input": "Find documentation on project management best practices"})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `TaskStatusLookup` with `{'task_id': 'task1'}`


In ProgressThe status of task1 is **In Progress**.

> Finished chain.
The status of task1 is **In Progress**.


> Entering new AgentExecutor chain...

Invoking: `DocsSearch` with `{'query': 'project management best practices'}`


snippet: Here are some top project management best practices shared by these seasoned professionals: Clear Project Objectives: According to experts, clearly defined project objectives are essential. A clear understanding of what must be achieved helps align the team and stakeholders., title: PROJECT MANAGEMENT BEST PRACTICES | Mackenny Sello..., link: https://www.linkedin.com/posts/mackenny-sello-mafako-9a0422a0_project-management-best-practices-project-activity-7328771862216200192-q0WA, snippet: Risk management Manage risks alongside project plans for better execution. Task management Track progress, costs and hours on tasks and subtasks. Agile project manageme